<a href="https://colab.research.google.com/github/srirampattabiraman/Computer-vision/blob/master/S4/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function 
import torch
import torch.nn as nn           ##Base class for all neural network modules      
import torch.nn.functional as F ##For convolution functions (includes activation as well)
import torch.optim as optim   ## Caters optimization algorithms
from torchvision import datasets, transforms

In [0]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -1 OUtput - 16 ##input channel, #outputchannel, kernelsize
#         self.bn1   = nn.BatchNorm2d(16)
#         self.d1    = nn.Dropout2d(0.1)
#         self.conv2 = nn.Conv2d(16, 32, 3, padding=1) #input_channel_size - 16, output_channel_size - 32,  kernel_size = 3 RF=5
#         self.bn2   = nn.BatchNorm2d(32)
#         self.d2    = nn.Dropout2d(0.1)
#         self.pool1 = nn.MaxPool2d(2, 2) ## Max pooling of 2*2 hence Receptive field doubles RF=10
# ###
#         self.conv3 = nn.Conv2d(32, 10, 3, padding=1) #input_channel_size - 32, output_channel_size - 10,  kernel_size = 3 
#         self.conv4 = nn.Conv2d(10, 16, 3, padding=1) #input_channel_size - 10, output_channel_size - 16,  kernel_size = 3 
#         self.bn3   = nn.BatchNorm2d(16)
#         self.d3    = nn.Dropout2d(0.1)
#         self.pool2 = nn.MaxPool2d(2, 2) ## Max pooling of 2*2 hence Receptive field doubles RF=28
# ###
#         self.conv5 = nn.Conv2d(16, 10, 3) ## #input_channel_size - 16, output_channel_size - 10
#         self.conv6 = nn.Conv2d(10, 16, 3) #input_channel_size - 10, output_channel_size - 16
#         self.bn4   = nn.BatchNorm2d(16)
#         self.d4    = nn.Dropout2d(0.1)
#         self.conv7 = nn.Conv2d(16, 10, 3) #input_channel_size - 1024, output_channel_size - 10,  kernel_size = 3 RF=34

#     def forward(self, x):
#         x = self.pool1(self.d2(self.bn2(F.relu(self.conv2(self.d1(self.bn1(F.relu(self.conv1(x)))))))))
#         x = self.pool2(self.d3(self.bn3(F.relu(self.conv4(F.relu(self.conv3(x)))))))
#         x = self.d4(self.bn4(F.relu(self.conv6(F.relu(self.conv5(x))))))
#         x = F.relu(self.conv7(x))
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1 = nn.Sequential(
                                nn.Conv2d(1,16,3),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout2d(0.1),
                                nn.Conv2d(16,32,3),nn.ReLU(),nn.BatchNorm2d(32),nn.Dropout2d(0.1),
                                nn.Conv2d(32,10,3),nn.ReLU(),#nn.BatchNorm2d(128),nn.Dropout2d(0.05),
                                nn.MaxPool2d(2,2)
                                )

    self.conv2 = nn.Sequential(
                                nn.Conv2d(10,16,3),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout2d(0.1),
                                nn.Conv2d(16,16,3),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout2d(0.1),
                                nn.Conv2d(16,16,3),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout2d(0.1)
                                )
    
    self.conv3 = nn.Sequential(
                                nn.Conv2d(16,16,3),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout2d(0.1),
                                nn.Conv2d(16,10,3),nn.ReLU()
                                
                                )

  def forward(self,x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)

    x = x.view(-1,10)

    return F.log_softmax(x)

In [35]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,640
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
         Dropout2d-8           [-1, 32, 24, 24]               0
            Conv2d-9           [-1, 10, 22, 22]           2,890
             ReLU-10           [-1, 10, 22, 22]               0
        MaxPool2d-11           [-1, 10, 11, 11]               0
           Conv2d-12             [-1, 16, 9, 9]           1,456
             ReLU-13             [-1, 16, 9, 9]               0
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:

torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [39]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)




  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



loss=2.4116427898406982 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]


loss=2.4116427898406982 batch_id=0:   0%|          | 1/469 [00:00<00:54,  8.62it/s]


loss=2.363346815109253 batch_id=1:   0%|          | 1/469 [00:00<00:54,  8.62it/s] 


loss=2.392430305480957 batch_id=2:   0%|          | 1/469 [00:00<00:54,  8.62it/s]


loss=2.270756959915161 batch_id=3:   0%|          | 1/469 [00:00<00:54,  8.62it/s]


loss=2.2744719982147217 batch_id=4:   0%|          | 1/469 [00:00<00:54,  8.62it/s]


loss=2.2744719982147217 batch_id=4:   1%|          | 5/469 [00:00<00:42, 10.90it/s]


loss=2.234381914138794 batch_id=5:   1%|          | 5/469 [00:00<00:42, 10.90it/s] 


loss=2.1951892375946045 batch_id=6:   1%|          | 5/469 [00:00<00:42, 10.90it/s]


l


Test set: Average loss: 0.0542, Accuracy: 9835/10000 (98%)






loss=0.138729065656662 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]


loss=0.1145838052034378 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]


loss=0.1145838052034378 batch_id=1:   0%|          | 2/469 [00:00<00:29, 15.96it/s]


loss=0.06326349079608917 batch_id=2:   0%|          | 2/469 [00:00<00:29, 15.96it/s]


loss=0.06282559037208557 batch_id=3:   0%|          | 2/469 [00:00<00:29, 15.96it/s]


loss=0.06504008919000626 batch_id=4:   0%|          | 2/469 [00:00<00:29, 15.96it/s]


loss=0.06504008919000626 batch_id=4:   1%|          | 5/469 [00:00<00:25, 18.41it/s]


loss=0.07902953773736954 batch_id=5:   1%|          | 5/469 [00:00<00:25, 18.41it/s]


loss=0.20666006207466125 batch_id=6:   1%|          | 5/469 [00:00<00:25, 18.41it/s]


loss=0.1181790828704834 batch_id=7:   1%|          | 5/469 [00:00<00:25, 18.41it/s] 


loss=0.1181790828704834 batch_id=7:   2%|▏         | 8/469 [00:00<00:22, 20.69it/s]


loss=0.1240677759051323 batch_id=8:   2%|▏         | 8/469 [0


Test set: Average loss: 0.0442, Accuracy: 9848/10000 (98%)






loss=0.06205245107412338 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]


loss=0.08826003968715668 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]


loss=0.08826003968715668 batch_id=1:   0%|          | 2/469 [00:00<00:27, 17.08it/s]


loss=0.08177655935287476 batch_id=2:   0%|          | 2/469 [00:00<00:27, 17.08it/s]


loss=0.04665321856737137 batch_id=3:   0%|          | 2/469 [00:00<00:27, 17.08it/s]


loss=0.17459014058113098 batch_id=4:   0%|          | 2/469 [00:00<00:27, 17.08it/s]


loss=0.06172802671790123 batch_id=5:   0%|          | 2/469 [00:00<00:27, 17.08it/s]


loss=0.06172802671790123 batch_id=5:   1%|▏         | 6/469 [00:00<00:23, 19.79it/s]


loss=0.07958964258432388 batch_id=6:   1%|▏         | 6/469 [00:00<00:23, 19.79it/s]


loss=0.08206726610660553 batch_id=7:   1%|▏         | 6/469 [00:00<00:23, 19.79it/s]


loss=0.020552676171064377 batch_id=8:   1%|▏         | 6/469 [00:00<00:23, 19.79it/s]


loss=0.07793854176998138 batch_id=9:   1%|▏         | 6

...


Test set: Average loss: 0.0228, Accuracy: 9923/10000 (99%)






loss=0.007716890424489975 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]


loss=0.007716890424489975 batch_id=0:   0%|          | 1/469 [00:00<00:49,  9.45it/s]


loss=0.010848041623830795 batch_id=1:   0%|          | 1/469 [00:00<00:49,  9.45it/s]


loss=0.03673042729496956 batch_id=2:   0%|          | 1/469 [00:00<00:49,  9.45it/s] 


loss=0.019770152866840363 batch_id=3:   0%|          | 1/469 [00:00<00:49,  9.45it/s]


loss=0.017175322398543358 batch_id=4:   0%|          | 1/469 [00:00<00:49,  9.45it/s]


loss=0.017175322398543358 batch_id=4:   1%|          | 5/469 [00:00<00:38, 12.11it/s]


loss=0.01643875613808632 batch_id=5:   1%|          | 5/469 [00:00<00:38, 12.11it/s] 


loss=0.009454634040594101 batch_id=6:   1%|          | 5/469 [00:00<00:38, 12.11it/s]


loss=0.04865717515349388 batch_id=7:   1%|          | 5/469 [00:00<00:38, 12.11it/s] 


loss=0.04865717515349388 batch_id=7:   2%|▏         | 8/469 [00:00<00:31, 14.69it/s]


loss=0.09073010087013245 batch_id=8:  


Test set: Average loss: 0.0232, Accuracy: 9922/10000 (99%)






loss=0.03282288461923599 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]


loss=0.03282288461923599 batch_id=0:   0%|          | 1/469 [00:00<00:49,  9.54it/s]


loss=0.017633553594350815 batch_id=1:   0%|          | 1/469 [00:00<00:49,  9.54it/s]


loss=0.07227494567632675 batch_id=2:   0%|          | 1/469 [00:00<00:49,  9.54it/s] 


loss=0.0072192177176475525 batch_id=3:   0%|          | 1/469 [00:00<00:49,  9.54it/s]


loss=0.010438375174999237 batch_id=4:   0%|          | 1/469 [00:00<00:49,  9.54it/s] 


loss=0.010438375174999237 batch_id=4:   1%|          | 5/469 [00:00<00:38, 12.14it/s]


loss=0.0233948715031147 batch_id=5:   1%|          | 5/469 [00:00<00:38, 12.14it/s]  


loss=0.0313178114593029 batch_id=6:   1%|          | 5/469 [00:00<00:38, 12.14it/s]


loss=0.017465665936470032 batch_id=7:   1%|          | 5/469 [00:00<00:38, 12.14it/s]


loss=0.035323768854141235 batch_id=8:   1%|          | 5/469 [00:00<00:38, 12.14it/s]


loss=0.035323768854141235 batch_id=8:  


Test set: Average loss: 0.0214, Accuracy: 9930/10000 (99%)

